In [ ]:
import nrrd
from scipy import ndimage
from skimage.exposure import rescale_intensity
import tifffile as tiff
import numpy as np
from scipy import interpolate 
import os 
import glob

p='E:\\i\\20230316\\'


In [ ]:
def detect_bidi_offset(plane, maxoff=5, zrange=(0, -1)):
    '''
    # Function written by Joe Donovan for bidirectional scan offsets correction
    :param plane:
    :return:
    '''

    aplane = plane[zrange[0]:zrange[1]].mean(0)

    offsets = np.arange(-maxoff+1, maxoff)
    x = np.arange(aplane.shape[-1])
    minoff = []
    for row in range(2, plane.shape[-2]-1, 2):
        f = interpolate.interp1d(x, (aplane[row + 1, :] + aplane[row - 1, :]) * .5, fill_value='extrapolate')

        offsetscores = np.asarray([np.mean(np.abs(aplane[row, :] - f(x + offset))) for offset in offsets])
        minoff.append(offsetscores.argmin())
    bestoffset = offsets[np.bincount(minoff).argmax()]
    print(bestoffset)
    return bestoffset

def bidi_offset_correction_plane(plane, offset=None, maxoff=5, zrange=(0, -1)):
    if not offset:
        offset = detect_bidi_offset(plane, maxoff=maxoff, zrange=zrange)
    plane[:, 1::2, :] = np.roll(plane[:, 1::2, :], -offset, -1)  # .astype(aplane.dtype)
    return

def generate_nrrd(p,rot_angle=135, date=20230316):

    fList=glob.glob(p+'*.tif')
    print(fList)

    for fno,f in enumerate(fList):
        
        zstack = tiff.imread(f, )
        if len(zstack.shape) == 4:
            print('Reshaping ZSTACK!', zstack.shape)
            zstack = np.reshape(zstack, (zstack.shape[0]*zstack.shape[1], zstack.shape[2], zstack.shape[3]))

        if 'bidicorr' not in f:
            bidi_offset_correction_plane(zstack, maxoff=20, zrange=(int(zstack.shape[0]/2), int(zstack.shape[0]/2)+10))
        zstack = rescale_intensity(zstack,
                                   in_range=(
                                   np.percentile(zstack, 0), np.percentile(zstack, 99.99)), # could be optimized to use as much bit depth as possible
                                   out_range=(0, 255))
        zstack = zstack.astype(np.uint8)

        zstack_rot = ndimage.rotate(zstack, axes=(1, 2), angle=rot_angle, reshape=True, cval=0)
        zfile_tiff = os.path.join(f[:-4]+'preProcessed.tif')
        tiff.imwrite(zfile_tiff, zstack_rot)

         #adjust to match the scaling of this brain to the reference brain.
          #  e.g. 2 for animals > 16dpf, vs. 1 for younger
        # ideally, this should be the exact scale.
        #unit is microns per pixel.

        ageScale=2

        sd = np.array([
            [ageScale, 0, 0],
            [0, ageScale, 0],
            [0, 0, ageScale]]).astype(float)



        header = {
            'space dimension': 3,
            'space directions': sd,
            'space units': ["microns", "microns", "microns"],
            'type': 'uint8',
            'PixelType': 'uint8'
        }
        zstack_rot = np.moveaxis(zstack_rot, 0, -1)
        zstack_rot = np.moveaxis(zstack_rot, 0, 1)

        zfile_nrrd = zfile_tiff.split('.')[0] + '.nrrd'
        nrrd.write(zfile_nrrd, zstack_rot, header=header)
    return True

In [ ]:
generate_nrrd(p)